Spam filter for SMS messages

In [1]:
import pandas as pd
SMSSpam = pd.read_csv('SMSSpamCollection',sep='\t',header=None, names=['Label','SMS'])


In [2]:
SMSSpam.Label.value_counts(normalize= True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

In [3]:
data = SMSSpam.sample(frac = 1, random_state=1)
training_set = data.iloc[:round(0.8*data.shape[0]),]
test_data = data.iloc[round(0.8*data.shape[0]):,]

In [4]:
p_ham = training_set.Label.value_counts(normalize =  True)[0]
p_spam = training_set.Label.value_counts(normalize =  True)[1]
test_data.Label.value_counts(normalize = True)
p_ham

0.8654104979811574

In [5]:
training_test_index = round(len(data) * 0.8)

# Training/Test split
training_set = data[:training_test_index].reset_index(drop=True)
training_set.head(5)

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [6]:

SMSS = SMSSpam.SMS.str.replace('\W',' ').str.lower()
SMSS

0       go until jurong point  crazy   available only ...
1                           ok lar    joking wif u oni   
2       free entry in 2 a wkly comp to win fa cup fina...
3       u dun say so early hor    u c already then say   
4       nah i don t think he goes to usf  he lives aro...
5       freemsg hey there darling it s been 3 week s n...
6       even my brother is not like to speak with me  ...
7       as per your request  melle melle  oru minnamin...
8       winner   as a valued network customer you have...
9       had your mobile 11 months or more  u r entitle...
10      i m gonna be home soon and i don t want to tal...
11      six chances to win cash  from 100 to 20 000 po...
12      urgent  you have won a 1 week free membership ...
13      i ve been searching for the right words to tha...
14                    i have a date on sunday with will  
15      xxxmobilemovieclub  to use your credit  click ...
16                             oh k   i m watching here  
17      eh u r

In [7]:
training_set.Label.value_counts()

ham     3858
spam     600
Name: Label, dtype: int64

In [8]:
SMSS.str.split(' ')
print(type(SMSS))
vocabulary = []
for i in range(0,len(SMSS)):
    vocabulary.append(SMSS[i])


<class 'pandas.core.series.Series'>


In [9]:
type(vocabulary)

list

In [10]:
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)
vocabulary

['i re met alex nichols from middle school and it turns out he s dealing ',
 'i dnt wnt to tlk wid u',
 'freemsg  our records indicate you may be entitled to 3750 pounds for the accident you had  to claim for free reply with yes to this msg  to opt out text stop',
 'if i was i wasn t paying attention',
 'pathaya enketa maraikara pa ',
 'dude we should go sup again',
 'v nice  off 2 sheffield tom 2 air my opinions on categories 2 b used 2 measure ethnicity in next census  busy transcribing     ',
 'you still around  looking to pick up later',
 'i m so in love with you  i m excited each day i spend with you  you make me so happy ',
 'why did i wake up on my own  gt   ',
 'also hi wesley how ve you been',
 'yes just finished watching days of our lives  i love it ',
 'no worries  hope photo shoot went well  have a spiffing fun at workage  ',
 'k do i need a login or anything',
 'it just seems like weird timing that the night that all you and g want is for me to come smoke is the same day a

In [11]:
training_set['SMS'] = training_set['SMS'].str.replace('\W',' ').str.lower()

training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [12]:
 
training_set['SMS'] = training_set['SMS'].str.split()

In [13]:
vocabulary  = []
for value in training_set['SMS']:
    for word in value:
        vocabulary.append(word)
vocabulary = list(set(vocabulary))

In [14]:
len(vocabulary)

7783

In [15]:
word_counts_per_sms = {unique_words : [0]*len(training_set['SMS']) for unique_words in vocabulary}


In [16]:
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] +=1

In [17]:
word_counts = pd.DataFrame(word_counts_per_sms)

In [18]:
training_set_clean = pd.concat([training_set,word_counts],axis = 1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [19]:
p_Spam = (training_set_clean.Label == 'spam').sum()/training_set_clean.shape[0]
p_Ham = (training_set_clean.Label == 'ham').sum()/training_set_clean.shape[0]
print(p_Spam)
print(p_Ham)


0.13458950201884254
0.8654104979811574


In [20]:
training_set_spam = training_set_clean[training_set_clean.Label == 'spam'].iloc[:,2:]
training_set_ham = training_set_clean[training_set_clean.Label == 'ham'].iloc[:,2:]
spam_total_words =0
ham_total_words = 0
for i in range (training_set_spam.shape[0]):
    row_words = training_set_spam.iloc[i].sum()
    spam_total_words += row_words
for i in range (training_set_ham.shape[0]):
    row_words = training_set_ham.iloc[i].sum()
    ham_total_words += row_words
print(spam_total_words)
print(ham_total_words)

15190
57237


In [21]:
alpha =1

In [22]:
spam = {unique_word : 0 for unique_word in vocabulary}
ham = {unique_word : 0 for unique_word in vocabulary}
wi_spam_occurs = training_set_spam.sum(axis = 0)
wi_ham_occurs = training_set_ham.sum(axis = 0)
for word in wi_spam_occurs.index:
    p_word = (wi_spam_occurs[word] +1)/(spam_total_words +len(vocabulary))
    spam[word] = p_word
for word in wi_ham_occurs.index:
    p_word = (wi_ham_occurs[word] +1)/(spam_total_words +len(vocabulary))
    ham[word] = p_word
    




In [23]:
p_word = (wi_spam_occurs['00'] +1)/(spam_total_words +len(vocabulary))
print(p_word)
spam['00']

0.0004352936055369347


0.0004352936055369347

In [24]:
spam

{'diff': 4.3529360553693465e-05,
 'favor': 4.3529360553693465e-05,
 '09050000301': 8.705872110738693e-05,
 'fuelled': 4.3529360553693465e-05,
 '08712400200': 8.705872110738693e-05,
 'big': 8.705872110738693e-05,
 'mmmm': 4.3529360553693465e-05,
 'hit': 0.00021764680276846734,
 'positive': 4.3529360553693465e-05,
 'gbp5': 8.705872110738693e-05,
 'oooooh': 4.3529360553693465e-05,
 'yupz': 4.3529360553693465e-05,
 'fuuuuck': 4.3529360553693465e-05,
 'other': 0.00021764680276846734,
 '021': 8.705872110738693e-05,
 'saibaba': 4.3529360553693465e-05,
 'cm2': 0.00017411744221477386,
 '786': 0.00021764680276846734,
 'thasa': 4.3529360553693465e-05,
 'gentle': 4.3529360553693465e-05,
 'activities': 4.3529360553693465e-05,
 'lag': 4.3529360553693465e-05,
 'head': 4.3529360553693465e-05,
 'sweetie': 4.3529360553693465e-05,
 'squeezed': 4.3529360553693465e-05,
 'shrek': 4.3529360553693465e-05,
 'deviousbitch': 4.3529360553693465e-05,
 'l8rs': 4.3529360553693465e-05,
 'reference': 0.000174117442214

In [25]:
import re
def classify(message):
    message = re.sub('\W','',message)
    message = message.lower().split()
    p_spam_given_message = p_spam
    p_ham_given_message =p_ham
    for word in message:
        if word in spam:
            p_spam_given_message *= spam[word]
        if word in ham:
            p_ham_given_message *= ham[word]
        if p_spam_given_message > p_ham_given_message:
            return 'spam'
        elif p_spam_given_message < p_ham_given_message:
            return 'ham'
        else: 
            return 'need human to classify'

In [26]:
p_spam

0.13458950201884254

In [27]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

'ham'

In [28]:
import re
def classify_test(message):
    message = re.sub('\W','',message)
    message = message.lower().split()
    p_spam_given_message = p_spam
    p_ham_given_message =p_ham
    for word in message:
        if word in spam:
            p_spam_given_message *= spam[word]
        if word in ham:
            p_ham_given_message *= ham[word]
        if p_spam_given_message > p_ham_given_message:
            return 'spam'
        elif p_spam_given_message < p_ham_given_message:
            return 'ham'
        else: 
            return 'need human to classify'
test_data['predicted'] = test_data['SMS'].apply(classify_test)
test_data.head()


/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Label,SMS,predicted
2131,ham,Later i guess. I needa do mcat study too.,ham
3418,ham,But i haf enuff space got like 4 mb...,ham
3424,spam,Had your mobile 10 mths? Update to latest Oran...,ham
1538,ham,All sounds good. Fingers . Makes it difficult ...,ham
5393,ham,"All done, all handed in. Don't know if mega sh...",ham


NameError: name 'training_data' is not defined